## **ADS CREATION V10.4**
v10.4.0 - Removed other event from the data-set, identified 4 major outages and removed them for outages, also used IQR: <br>
- Used IQR range 0.1 to 0.99 and removed data-set based on these ranges

v10.3.0 - Adding no of outages based on CLUE, CAUSE, OCCURN: <br>
- 'NO_OF_POWER_OUT_CLUE_PER_DAY', 'NO_OF_OPEN_DEVICE_CLUE_PER_DAY', 'NO_OF_IVR_CLUE_PER_DAY', 'NO_OF_ANIMAL_CAUSE_PER_DAY', 'NO_OF_WIRE_OCCURN_PER_DAY' <br>
- Categorized outages based on dispatch area location '34th', 'ARL.', 'MILL', 'ENGLISH', 'W.I.', 'SOUTH' and 'NO_LOCATION' based on minimum distance from outage <br>

v10.2.0 - Adding priority/queueing feature into the analytical dataset based on : <br>
- Rank based on simple customer quantity as mentioned by Eric (live rankings to be followed, numerical feature) <br>
- Rank based on the factor of distance from centroid and customer quantity (live rankings to be followed, approach #2, numerical feature) <br>

In [ ]:
import csv
import math
import time
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

from dateutil.parser import parse
from datetime import datetime
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

## **FILTERING FAC_JOB TABLE**

In [ ]:
%%time
df_facility_job_his = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_FACILITY_JOB_IPL.csv', sep = ",", encoding = "ISO-8859-1")

CPU times: user 34.9 s, sys: 4.45 s, total: 39.4 s
Wall time: 5min 3s


In [ ]:
# changing the date format

df_facility_job_his["CREATION_DATETIME"] = pd.to_datetime(df_facility_job_his["CREATION_DATETIME"], errors ='coerce')
df_facility_job_his["ENERGIZED_DATETIME"] = pd.to_datetime(df_facility_job_his["ENERGIZED_DATETIME"], errors ='coerce')
df_facility_job_his["ETR_DATETIME"] = pd.to_datetime(df_facility_job_his["ETR_DATETIME"], errors ='coerce')

# creating blue sky flags
df_facility_job_his['BLUE_SKY_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID == 0) | (df_facility_job_his.MAJ_OTG_ID.isnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME - 
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2002))
print("Total blue sky Events: ", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))

# creating storm event flags

df_facility_job_his['STORM_EVENT_FLG'] = ( (df_facility_job_his.TOT_LOSS_POWER_FLG == 'T') & 
                                           ((df_facility_job_his.MAJ_OTG_ID != 0) & (df_facility_job_his.MAJ_OTG_ID.notnull())) &
                                           ((df_facility_job_his.ISOLATED_TO_CUST_FLG == 'F') | (df_facility_job_his.ISOLATED_TO_CUST_FLG.isnull())) &
                                           ((df_facility_job_his.ROUTINE_FLG == 'F') | (df_facility_job_his.ROUTINE_FLG.isnull())) &
                                           ((df_facility_job_his.ENERGIZED_DATETIME -
                                             df_facility_job_his.CREATION_DATETIME).dt.total_seconds().div(60).round(2) > 5) 
                                          & (df_facility_job_his.CREATION_DATETIME.dt.year > 2002))
print("Total Storm Events: ", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))


# filtering data

occur_remov = [30003001, 33003301, 33003302, 34003400, 34003401, 34003402, 34003403, 34003404, 34003405, 34003406, 34003407, 34003408, 34003409, 35003500,
                35003501, 35003502, 35003503, 35003504, 35003505, 35003506, 35003507, 35003508, 36003600, 36003601, 36003602, 36003603, 36003604, 36003605,
                36003606, 36003607, 36003608, 37003703, 38003802, 38003803, 38003804, 38003807, 39003910, 41004100, 41004101, 41004102, 48004800, 48004802,
                48004803, 49004900, 49004901, 49004902, 50005000, 50005001, 50005002, 52005200, 52005201, 52005202, 52005203, 52005204, 52005205, 52005206,
                52005207, 53005300, 53005301, 53005302, 53005303, 53005304, 53005305, 53005306, 53005307, 53005308, 53005309, 53005310, 54005400, 54005401,
                54005402, 54005403, 54005404, 54005405, 34003410, 30003000, 36503650, 36503651, 36503652, 36503653, 36503654, 36503655, 36503656, 36503657,
                36503658]

init_rows = len(df_facility_job_his)
init_blue_sky = len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True])
init_storm = len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True])
init_incident = df_facility_job_his.INCIDENT_ID.nunique()
_incident_ = len(df_facility_job_his[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Initial Rows", init_rows)
print("Initial blue sky events", init_blue_sky)
print("Initial Storm events", init_storm)
print("Number of incident id", init_incident)
print("Unique incident-structure no",_incident_)

print("-------------After Filtering Creation_datetime from 2007 to 2019---------------------")
df_facility_job_his = df_facility_job_his[(df_facility_job_his.CREATION_DATETIME.dt.year > 2006) & (df_facility_job_his.CREATION_DATETIME.dt.year < 2020)]
_incident_ = len(df_facility_job_his[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_facility_job_his))
print("blue sky events", len(df_facility_job_his[df_facility_job_his.BLUE_SKY_FLG == True]))
print("Storm events", len(df_facility_job_his[df_facility_job_his.STORM_EVENT_FLG == True]))
print("Number of incident id", df_facility_job_his.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing CUST_QTY and DOWNSTREAM_CUST_QTY>0----------------------")
df_fac_f1 = df_facility_job_his[(df_facility_job_his.CUST_QTY > 0) & (df_facility_job_his.DOWNSTREAM_CUST_QTY > 0)]
_incident_ = len(df_fac_f1[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f1))
print("blue sky events", len(df_fac_f1[df_fac_f1.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f1[df_fac_f1.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f1.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Equip_stn_No as <NCC> and null----------------------")
df_fac_f2 = df_fac_f1[(df_fac_f1.EQUIP_STN_NO != '<NCC>')]
df_fac_f2 = df_fac_f2[~df_fac_f2.EQUIP_STN_NO.isnull()]
_incident_ = len(df_fac_f2[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f2))
print("blue sky events", len(df_fac_f2[df_fac_f2.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f2[df_fac_f2.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f2.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------")
df_fac_f3 = df_fac_f2[(df_fac_f2.OCCURN_DESC.str.find("CANCEL") == -1)]
df_fac_f3 = df_fac_f3[(df_fac_f3.OCCURN_DESC.str.find("FOUND OK") == -1)]
_incident_ = len(df_fac_f3[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f3))
print("blue sky events", len(df_fac_f3[df_fac_f3.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f3[df_fac_f3.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f3.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing NaN from DNI, Circuit ID 0 and NaN  from Structure Nos.----------------------")
df_fac_f4 = df_fac_f3[(df_fac_f3.CIRCT_ID != 0)]
df_fac_f4 = df_fac_f4[~df_fac_f4.CIRCT_ID.isnull()]
df_fac_f4 = df_fac_f4[~df_fac_f4.STRCTUR_NO.isnull()]
df_fac_f4 = df_fac_f4[~df_fac_f4.DNI_EQUIP_TYPE.isnull()]
_incident_ = len(df_fac_f4[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f4))
print("blue sky events", len(df_fac_f4[df_fac_f4.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f4[df_fac_f4.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f4.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing NaN Energized datetime ----------------------")
df_fac_f5 = df_fac_f4[~(df_fac_f4.ENERGIZED_DATETIME.isnull())]
_incident_ = len(df_fac_f5[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f5))
print("blue sky events", len(df_fac_f5[df_fac_f5.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f5[df_fac_f5.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f5.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Energized datetime < CREATION_DATETIME----------------------")
df_fac_f6 = df_fac_f5[df_fac_f5.ENERGIZED_DATETIME > df_fac_f5.CREATION_DATETIME]
_incident_ = len(df_fac_f6[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f6))
print("blue sky events", len(df_fac_f6[df_fac_f6.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f6[df_fac_f6.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f6.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)
                       
print("-------------After removing CLUE_CD not starting with zero----------------------")
df_fac_f7 = df_fac_f6[(df_fac_f6.CLUE_CD.str[:1] == '0') & (df_fac_f6.CLUE_CD.str[:2] != '00')]
df_fac_f7 = df_fac_f7[df_fac_f7.CLUE_CD != '01']
df_fac_f7 = df_fac_f7[df_fac_f7.CLUE_CD != '09OD']
_incident_ = len(df_fac_f7[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f7))
print("blue sky events", len(df_fac_f7[df_fac_f7.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f7[df_fac_f7.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f7.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

print("-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------")
df_fac_f8 = df_fac_f7[~(df_fac_f7.OCCURN_CD.isin(occur_remov))]
_incident_ = len(df_fac_f8[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("Rows", len(df_fac_f8))
print("blue sky events", len(df_fac_f8[df_fac_f8.BLUE_SKY_FLG == True]))
print("Storm events", len(df_fac_f8[df_fac_f8.STORM_EVENT_FLG == True]))
print("Number of incident id", df_fac_f8.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

# Incident_id to be removed 1750 cases

exclude_inc_ = pd.read_csv(r'gs://aes-analytics-0002-curated/Outage_Restoration/Test/EXCLUDE_INC.csv', header=None)[1]

df_fac_final = df_fac_f8[~(df_fac_f8.INCIDENT_ID.isin(exclude_inc_))]

#Changing Flags to 0 and 1 
df_fac_final.BLUE_SKY_FLG = df_fac_final.BLUE_SKY_FLG.apply(lambda x: 1 if x==True else 0) 
df_fac_final.STORM_EVENT_FLG = df_fac_final.STORM_EVENT_FLG.apply(lambda x: 1 if x==True else 0) 

del df_facility_job_his, df_fac_f1, df_fac_f2, df_fac_f3, df_fac_f4, df_fac_f5, df_fac_f6, df_fac_f7

Total blue sky Events:  139201
Total Storm Events:  39071
Initial Rows 1268993
Initial blue sky events 139201
Initial Storm events 39071
Number of incident id 914650
Unique incident-structure no 1021773
-------------After Filtering Creation_datetime from 2007 to 2019---------------------
Rows 937870
blue sky events 107094
Storm events 33036
Number of incident id 832388
Unique structure no 872966
-------------After removing CUST_QTY and DOWNSTREAM_CUST_QTY>0----------------------
Rows 627436
blue sky events 96564
Storm events 32789
Number of incident id 540420
Unique structure no 577570
-------------After removing Equip_stn_No as <NCC> and null----------------------
Rows 610810
blue sky events 96262
Storm events 32730
Number of incident id 533462
Unique structure no 570331
-------------After removing Occurence code starting with Cancel, Found OK and duplicate ----------------------
Rows 418252
blue sky events 96213
Storm events 32676
Number of incident id 372606
Unique structure no 3883

In [ ]:
print("Rows", len(df_fac_final))
_incident_ = len(df_fac_final[['INCIDENT_ID','STRCTUR_NO']].drop_duplicates())
print("blue sky events", len(df_fac_final[df_fac_final.BLUE_SKY_FLG == 1]))
print("Storm events", len(df_fac_final[df_fac_final.STORM_EVENT_FLG == 1]))
print("Number of incident id", df_fac_final.INCIDENT_ID.nunique())
print("Unique structure no",_incident_)

Rows 107313
blue sky events 60647
Storm events 23224
Number of incident id 86969
Unique structure no 94420


In [ ]:
df_maj_otg_df = df_fac_final[(~df_fac_final.MAJ_OTG_ID.isnull()) & (df_fac_final.MAJ_OTG_ID != 0)]
maj_otg_df = df_maj_otg_df.groupby(['INCIDENT_ID', 'STRCTUR_NO'], as_index = False).agg({'MAJ_OTG_ID' : pd.Series.unique})
inc_ = maj_otg_df[maj_otg_df.MAJ_OTG_ID>1].INCIDENT_ID

In [ ]:
x = df_fac_final[df_fac_final.INCIDENT_ID.isin(inc_)]

In [ ]:
maj_otg_df = df_maj_otg_df.groupby(['INCIDENT_ID', 'STRCTUR_NO'], as_index = False).agg({'MAJ_OTG_ID' : 'max'})

In [ ]:
df_event_flg = df_fac_final[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID' ,'DNI_EQUIP_TYPE','STORM_EVENT_FLG']]
def event_flag(group):
    group = group.reset_index(drop = True)
    if(group.STORM_EVENT_FLG.sum() >=1):
        group['EVENT'] = 'STORM'
        return group
    else:
        group['EVENT'] = 'BLUE SKY'
        return group
df_event_flg = df_event_flg.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).apply(event_flag).reset_index(drop = True)

In [ ]:
df_check = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE']).nunique()
df_check.sum()

FAC_JOB_ID              107313
MAJ_OTG_ID               90727
EQUIP_STN_NO            107292
DIST_NO                  94461
HOST_SEQ_ID                  0
PRIORITY_VAL             96885
CUST_QTY                105973
CLUE_CD                  99806
CLUE_DESC                99806
CREATION_DATETIME        94461
CALL_QTY                104769
KEY_CUST_QTY             99511
SPLIT_FAC_JOB_FLG        41536
CAUSE_CD                 91721
CAUSE_DESC               91700
OCCURN_CD                94460
OCCURN_DESC              94629
CLIMATIC_CD              81240
CLIMATIC_DESC            81240
CITY_NAM                 93432
LOC_DESC                 92717
WRK_ORD_NUM                  0
COMMENT_TEXT             34620
CALL_ID                 107313
KVA_VAL                 104469
BOOK_NO                      0
ADDRESS                 104748
CIRCT_NAM                94461
CLUE_CD2                  1897
INSERTED_DATE            95051
DOWNSTREAM_KVA_VAL      105084
DOWNSTREAM_CUST_QTY     105973
COMPL_DA

In [ ]:
df_numerical = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ], as_index = False).agg({'CUST_QTY':'sum','CALL_QTY':'sum','KEY_CUST_QTY':'sum','DOWNSTREAM_CUST_QTY':'sum','KVA_VAL':'mean',
                                       'DOWNSTREAM_KVA_VAL':'mean', 'FAC_JOB_ID': 'max', 'ETR_DATETIME': 'max', 'CREATION_DATETIME': 'min', 'MAJ_OTG_ID' : 'sum',
                                      'ENERGIZED_DATETIME': 'max', 'SUBST_ID': 'min', 'COMMENT_TEXT' : 'last'})

In [ ]:
df_numerical = pd.merge(df_event_flg, df_numerical, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE' ])

In [ ]:
df_numerical.drop_duplicates(keep='first', inplace=True)

In [ ]:
print(df_numerical.shape)

(94523, 19)


## **Adding extra columns like Day Night flag and creating dependent variable TTR column**

In [1]:
# creating day night flag for outages

df_numerical['DAY_FLAG'] = df_numerical.CREATION_DATETIME.dt.hour.apply(lambda x: 1 if ((x >= 6) & (x<18)) else 0)
df_numerical['TTR'] = (df_numerical.ENERGIZED_DATETIME - df_numerical.CREATION_DATETIME).dt.total_seconds().div(60).round(4)

NameError: ignored

In [ ]:
print(df_numerical.shape)

## **CLUE CODE CLEAN**

In [ ]:
df_fac_final.PRIORITY_VAL = df_fac_final.PRIORITY_VAL.astype(float)

df_fac_final['PRIORITY_VAL_1.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 1 else 0)
df_fac_final['PRIORITY_VAL_2.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 2 else 0)
df_fac_final['PRIORITY_VAL_3.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 3 else 0)
df_fac_final['PRIORITY_VAL_5.0'] = df_fac_final['PRIORITY_VAL'].apply(lambda x: 1 if x == 5 else 0)

df_fac_final.drop(['PRIORITY_VAL'],axis=1,inplace=True)

df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'INDIANAPOLIS' if(str(x).find('INDIAN') != -1) else x)
df_fac_final.CITY_NAM = df_fac_final.CITY_NAM.apply(lambda x: 'NO_CITY' if(x != x) else x)

In [ ]:
# segregation of clue code desc

df_fac_final['POLE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('pole') != -1) else 0)
df_fac_final['PART_LIGHT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('part lights') != -1) else 0)
df_fac_final['EMERGENCY_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('emergency') != -1) else 0)
df_fac_final['POWER_OUT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('power out') != -1) else 0)
df_fac_final['TREE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('tree') != -1) else 0)
df_fac_final['WIRE_DOWN_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('wire down') != -1) else 0)
df_fac_final['IVR_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.lower().find('ivr') != -1) else 0)
df_fac_final['EQUIPMENT_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('EQUIPMENT') != -1) else 0)
df_fac_final['TRANSFORMER_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('TRANSFORMER') != -1) else 0)
df_fac_final['OPEN_DEVICE_CLUE_FLG'] = df_fac_final.CLUE_DESC.apply(lambda x: 1 if (x.find('OPEN DEVICE') != -1) else 0)


#segration of cause desc
df_fac_final['CAUSE_DESC1'] = df_fac_final[['CAUSE_DESC']].fillna('0')
df_fac_final['OH_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('OH') != -1) | (x.find('O.H.') != -1)) else 0)
df_fac_final['UG_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if((x.find('UG') != -1) | (x.find('U.G.') != -1)) else 0)
df_fac_final['ANIMAL_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ANIMAL') != -1) else 0)
df_fac_final['WEATHER_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WEATHER') != -1) else 0)
df_fac_final['WEATHER_COLD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('COLD') != -1) else 0)
df_fac_final['WEATHER_LIGHTNING_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('LIGHTNING') != -1) else 0)
df_fac_final['WEATHER__SNOW_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SNOW') != -1) else 0)
df_fac_final['WEATHER__WIND_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('WIND') != -1) else 0)
df_fac_final['WEATHER__HEAT_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('HEAT') != -1) else 0)
df_fac_final['WEATHER__FLOOD_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('FLOOD') != -1) else 0)
df_fac_final['PUBLIC_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PUBLIC') != -1) else 0)
df_fac_final['STREET_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('ST ') != -1) else 0)
df_fac_final['SUBSTATION_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('SUBSTATION') != -1) else 0)
df_fac_final['TREE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('TREE') != -1) else 0)
df_fac_final['MISCELLANEOUS_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('MISCELLANEOUS') != -1) else 0)
df_fac_final['CUST_REQUEST_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('CUSTOMER REQUEST') != -1) else 0)
df_fac_final['NO_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO CAUSE') != -1) else 0)
df_fac_final['PLANNED_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('PLANNED WORK') != -1) else 0)
df_fac_final['NO_OUTAGE_CAUSE_FLG'] = df_fac_final.CAUSE_DESC1.apply(lambda x: 1 if(x.find('NO OUTAGE') != -1) else 0)


#segration of OCCURN desc
df_fac_final['FUSE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FUSE') != -1) & (x.find('FUSE NOT') == -1)) else 0)
df_fac_final['CUST_EQUIP_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUSTOMER EQUIP') != -1) else 0)
df_fac_final['POLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('POLE') != -1) else 0)
df_fac_final['TRANSFORMER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('TRANSFORMER') != -1) else 0)
df_fac_final['METER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('METER') != -1) else 0)
df_fac_final['SERVICE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SERVICE') != -1) else 0)
df_fac_final['CABLE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CABLE') != -1) else 0)
df_fac_final['ST_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('ST') != -1) else 0)
df_fac_final['FIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('FIRE') != -1) else 0)
df_fac_final['FOUND_OPEN_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if((x.find('FOUND OPEN') != -1) & (x.find('NOT FOUND OPEN') == -1)) else 0)
df_fac_final['PUBLIC_SAFETY_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SAFETY') != -1) else 0)
df_fac_final['WIRE_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('WIRE') != -1) else 0)
df_fac_final['SWITCH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('SWITCH') != -1) else 0)
df_fac_final['CUTOUT_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CUTOUT') != -1) else 0)
df_fac_final['REGULATOR_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('REGULATOR') != -1) else 0)
df_fac_final['CAP_BANK_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('CAP BANK') != -1) else 0)
df_fac_final['OH_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('OH') != -1) else 0)
df_fac_final['RECLOSER_OCCURN_FLG'] = df_fac_final.OCCURN_DESC.apply(lambda x: 1 if(x.find('RECLOSER') != -1) else 0)

df_fac_final = df_fac_final.drop(columns = ['CAUSE_DESC1'])

In [ ]:
df_fac_cat = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'POLE_CLUE_FLG': 'sum', 'PART_LIGHT_CLUE_FLG': 'sum',
                                                                          'EMERGENCY_CLUE_FLG': 'sum','POWER_OUT_CLUE_FLG': 'sum',
                                                                          'TREE_CLUE_FLG': 'sum', 'WIRE_DOWN_CLUE_FLG': 'sum',
                                                                          'OPEN_DEVICE_CLUE_FLG':'sum', 'EQUIPMENT_CLUE_FLG': 'sum',
                                                                          'TRANSFORMER_CLUE_FLG':'sum','IVR_CLUE_FLG': 'sum',
                                                                          'OH_CAUSE_FLG': 'sum', 'UG_CAUSE_FLG': 'sum', 
                                                                          'ANIMAL_CAUSE_FLG': 'sum','WEATHER_CAUSE_FLG': 'sum', 
                                                                          'WEATHER_COLD_CAUSE_FLG': 'sum','PUBLIC_CAUSE_FLG': 'sum',
                                                                         'WEATHER_LIGHTNING_CAUSE_FLG': 'sum', 'WEATHER__SNOW_CAUSE_FLG': 'sum',
                                                                          'WEATHER__WIND_CAUSE_FLG': 'sum','WEATHER__HEAT_CAUSE_FLG': 'sum',
                                                                         'WEATHER__FLOOD_CAUSE_FLG': 'sum', 'STREET_CAUSE_FLG': 'sum',
                                                                        'MISCELLANEOUS_CAUSE_FLG':'sum', 'CUST_REQUEST_CAUSE_FLG': 'sum',
                                                                          'SUBSTATION_CAUSE_FLG': 'sum','TREE_CAUSE_FLG': 'sum',
                                                                          'NO_CAUSE_FLG': 'sum', 'PLANNED_CAUSE_FLG': 'sum',
                                                                          'NO_OUTAGE_CAUSE_FLG': 'sum',
                                                                          'PRIORITY_VAL_1.0' : 'sum', 'PRIORITY_VAL_2.0': 'sum', 
                                                                          'PRIORITY_VAL_3.0': 'sum', 'PRIORITY_VAL_5.0': 'sum',
                                                                          'FUSE_OCCURN_FLG': 'sum', 'CUST_EQUIP_OCCURN_FLG': 'sum',
                                                                          'POLE_OCCURN_FLG': 'sum', 'TRANSFORMER_OCCURN_FLG': 'sum', 
                                                                          'METER_OCCURN_FLG': 'sum', 'SERVICE_OCCURN_FLG': 'sum',
                                                                          'CABLE_OCCURN_FLG': 'sum', 'ST_OCCURN_FLG': 'sum',
                                                                          'FIRE_OCCURN_FLG': 'sum', 'FOUND_OPEN_OCCURN_FLG': 'sum',
                                                                          'PUBLIC_SAFETY_OCCURN_FLG': 'sum', 'WIRE_OCCURN_FLG': 'sum',
                                                                          'SWITCH_OCCURN_FLG': 'sum', 'REGULATOR_OCCURN_FLG': 'sum',
                                                                          'CUTOUT_OCCURN_FLG': 'sum','CAP_BANK_OCCURN_FLG': 'sum',
                                                                          'RECLOSER_OCCURN_FLG': 'sum','OH_OCCURN_FLG': 'sum',
                                                                          'OCCURN_DESC' : 'last',
                                                                          'CAUSE_DESC': 'last', 'CLUE_DESC': 'last'
                                                                          })
dummy_col = ['POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
             'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0']
for i in dummy_col:
    df_fac_cat[i] =  df_fac_cat[i].apply(lambda x: 1 if x>=1 else 0)

df_fac_cat = df_fac_cat[['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE',
                         'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CAUSE_DESC', 'CLUE_DESC', 'OCCURN_DESC']].drop_duplicates()

In [ ]:
print(df_fac_cat.shape)

(94461, 58)


## **CAUSE and OCCURN DESC CLEANING**

In [ ]:
df_fac_final['OCCURN_DESC'] = df_fac_final['OCCURN_DESC'].str.replace(";","")

In [ ]:
t = df_fac_final.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).agg({'OCCURN_DESC': pd.Series.nunique,'CAUSE_DESC': pd.Series.nunique})

In [ ]:
t[t.OCCURN_DESC > 1] , t[t.CAUSE_DESC > 1]

(Empty DataFrame
 Columns: [INCIDENT_ID, STRCTUR_NO, CIRCT_ID, DNI_EQUIP_TYPE, OCCURN_DESC, CAUSE_DESC]
 Index: [], Empty DataFrame
 Columns: [INCIDENT_ID, STRCTUR_NO, CIRCT_ID, DNI_EQUIP_TYPE, OCCURN_DESC, CAUSE_DESC]
 Index: [])

In [ ]:
# df_fac_final[df_fac_final.INCIDENT_ID == t[t.OCCURN_DESC > 1].INCIDENT_ID[59012]].CAD_ID

In [ ]:
# del row with CAD_ID not present

# df_fac_final = df_fac_final.drop(689794)

In [ ]:
df_fac_final[df_fac_final.INCIDENT_ID.isin(t[t.CAUSE_DESC > 1].INCIDENT_ID)]

,FAC_JOB_ID,CIRCT_ID,MAJ_OTG_ID,EQUIP_STN_NO,DIST_NO,HOST_SEQ_ID,CUST_QTY,CLUE_CD,CLUE_DESC,CREATION_DATETIME,CALL_QTY,KEY_CUST_QTY,SPLIT_FAC_JOB_FLG,CAUSE_CD,CAUSE_DESC,OCCURN_CD,OCCURN_DESC,CLIMATIC_CD,CLIMATIC_DESC,CITY_NAM,LOC_DESC,WRK_ORD_NUM,COMMENT_TEXT,CALL_ID,KVA_VAL,BOOK_NO,ADDRESS,CIRCT_NAM,CLUE_CD2,INSERTED_DATE,DOWNSTREAM_KVA_VAL,DOWNSTREAM_CUST_QTY,COMPL_DATETIME,TOT_LOSS_POWER_FLG,ISOLATED_TO_CUST_FLG,PLANNED_OUTAGE_FLG,ROUTINE_FLG,DNI_EQUIP_TYPE,SUBST_ID,WORK_ORD_1_NO,WORK_ORD_2_NO,WORK_ORD_3_NO,WORK_ORD_4_NO,WORK_ORD_5_NO,ENERGIZED_DATETIME,DISPLAY_TEXT,POLICE_OPERATOR_ID,POLICE_INC_NO,FIRE_OPERATOR_ID,FIRE_INC_NO,CAD_ID,STRCTUR_NO,FAC_JOB_PARENT_ID,MAJ_INCIDENT_FLG,MAJ_INCIDENT_CAUSE,ZONE_DESC,DIST_DESC,ZONE_ID,GEO_DIST_NO,ETR_DATETIME,SUBST_SHUTDOWN_FLG,HIS_FAC_JOB_COMMENT,CIRCT_DESC,SUBST_DESC,DCNID_VAL,CREW_REMARKS,INCIDENT_ID,CLUE_DESC2,DIST_ABBREV,BLUE_SKY_FLG,STORM_EVENT_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,PUBLIC_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,CUTOUT_OCCURN_FLG,REGULATOR_OCCURN_FLG,CAP_BANK_OCCURN_FLG,OH_OCCURN_FLG,RECLOSER_OCCURN_FLG


In [ ]:
# #del row with CAD_ID not present
# df_fac_final = df_fac_final.drop(966712)
# # #completion time not present
# df_fac_final = df_fac_final.drop(966713)
# df_fac_final = df_fac_final.drop(477581)

In [ ]:
df_desc = df_fac_final[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE','OCCURN_DESC','CAUSE_DESC']].drop_duplicates()
df_desc = df_desc.fillna(0)

In [ ]:
split_desc = df_desc['OCCURN_DESC'].str.split('\\',expand=True)
df_desc['PRIM_OCCURN_DESC'] = split_desc[0]
split_desc = split_desc[1].str.split(', ',expand=True)
df_desc['SECO_OCCURN_DESC'] = split_desc[0]
df_desc['TERT_OCCURN_DESC'] = split_desc[1] 
df_desc['PRIM_OCCURN_DESC'] = df_desc['PRIM_OCCURN_DESC'].str.strip()
df_desc['SECO_OCCURN_DESC'] = df_desc['SECO_OCCURN_DESC'].str.strip()
df_desc['TERT_OCCURN_DESC'] = df_desc['TERT_OCCURN_DESC'].str.strip()

In [ ]:
split_desc = df_desc['CAUSE_DESC'].str.split('\\',expand=True)
df_desc['PRIM_CAUSE_DESC'] = split_desc[0]
df_desc['SECO_CAUSE_DESC'] = split_desc[1]
df_desc = df_desc.drop(columns = ['CAUSE_DESC','OCCURN_DESC'])

### **CITY**

In [ ]:
%%time

# city treatment
def cat_city_treat(group):
    if(group.CITY_NAM.nunique() > 1):
        x = group[group.CITY_NAM != 'NO_CITY'].CITY_NAM.unique()
        group.CITY_NAM = x[0]
        return group
    else:
        return group
df_treated = df_fac_final[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']]
df_treated = df_treated.groupby(['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], as_index = False).apply(cat_city_treat)

CPU times: user 1min 53s, sys: 818 ms, total: 1min 54s
Wall time: 12min 31s


In [ ]:
len(df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'CITY_NAM']].drop_duplicates())

94461

QC check complete

In [ ]:
df_treated = df_treated[['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE','CITY_NAM']].drop_duplicates()
df_fac_cat = pd.merge(df_fac_cat, df_treated, on = ['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE'])
df_fac_cat  = pd.merge(df_fac_cat, df_desc, on = ['INCIDENT_ID','STRCTUR_NO','CIRCT_ID', 'DNI_EQUIP_TYPE'])

In [ ]:
print(df_fac_cat.shape)

(94461, 64)


In [ ]:
df_fac_cat.head()

,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CAUSE_DESC,CLUE_DESC,OCCURN_DESC,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC
0,2000383864.00,494-B/403,1252.00,FUSE,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,O.H. EQUIPMENT\BAD ARRESTOR,PART LIGHTS - DIM/BRIGHT\PART OUT (RESET BREAK...,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD ARRESTOR
1,2000383866.00,404-A/85,1958.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL
2,2000383867.00,675--/125,1701.00,3TBOH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,PUBLIC\VANDALISM,POWER OUT - CUSTOMER\OUTAGE\BUSINESS,NONE\NONE,INDIANAPOLIS,NONE,NONE,None,PUBLIC,VANDALISM
3,2000383871.00,465-A/25,1960.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,MISCELLANEOUS\CUSTOMER EQUIPMENT,DO NOT USE - IVR\POWER OUT,CUSTOMER EQUIP.\CUST. OWNED WIRE,INDIANAPOLIS,CUSTOMER EQUIP.,CUST. OWNED WIRE,None,MISCELLANEOUS,CUSTOMER EQUIPMENT
4,2000383873.00,491XA/60,2702.00,FUSE,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL


## **X Y Coordinate treatment**

In [ ]:
%%time
df_his_location = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_LOCATION_IPL.csv', sep = "|")

CPU times: user 1min 35s, sys: 12.2 s, total: 1min 48s
Wall time: 13min 43s


In [ ]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','GEO_X_COORD','GEO_Y_COORD','PRIMARY_LOC_FLG']].drop_duplicates()
df_his_location = df_his_location[df_his_location.INCIDENT_ID.isin(df_fac_final.INCIDENT_ID)]
df_his_location = df_his_location[df_his_location.STRCTUR_NO.isin(df_fac_final.STRCTUR_NO)]

### **Changing X Y coordinates to LAT and LONG**

In [ ]:
%%time
# function to convert geo_x, geo_y coordinate to lat, long

def change_to_loc(df):
    demnorthing = df.GEO_Y_COORD
    demeasting = df.GEO_X_COORD
    northing = float(demnorthing) * 0.3048
    easting = float(demeasting) * 0.3048
    om = (northing - 250000 + 4151863.7425) / 6367236.89768
    fo = om + (math.sin(om) * math.cos(om)) * (0.005022893948 + 0.000029370625 * math.pow(math.cos(om), 2) + 0.000000235059 * math.pow(math.cos(om), 4) + 0.000000002181 * math.pow(math.cos(om), 6))
    tf = math.sin(fo) / math.cos(fo)
    nf2 = 0.00673949677548 * math.pow(math.cos(fo), 2)
    rn = 0.9999666667 * 6378137 / math.pow((1 - 0.0066943800229034 * math.pow(math.sin(fo), 2)), 0.5)
    q = (easting - 100000) / rn
    b2 = -0.5 * tf * (1 + nf2)
    b4 = -(1 / 12) * (5 + (3 * math.pow(tf, 2)) + (nf2 * (1 - 9 * math.pow(tf, 2)) - 4 * math.pow(nf2, 2)))
    b6 = (1 / 360) * (61 + (90 * math.pow(tf, 2)) + (45 * math.pow(tf, 4)) + (nf2 * (46 - (252 * math.pow(tf, 2)) - (90 * math.pow(tf, 4)))))
    lat = fo + b2 * math.pow(q, 2) * (1 + math.pow(q, 2) * (b4 + b6 * math.pow(q, 2)))
    b3 = -(1 / 6) * (1 + 2 * math.pow(tf, 2) + nf2)
    b5 = (1 / 120) * (5 + 28 * math.pow(tf, 2) + 24 * math.pow(tf, 4) + nf2 * (6 + 8 * math.pow(tf, 2)))
    b7 = -(1 / 5040) * (61 + 662 * math.pow(tf, 2) + 1320 * math.pow(tf, 4) + 720 * math.pow(tf, 6))
    l = q * (1 + math.pow(q, 2) * (b3 + math.pow(q, 2) * (b5 + b7 * math.pow(q, 2))))
    lon = 1.4951653925 - l / math.cos(fo)
    coord = [(lat * 57.2957795131), (-1 * lon * 57.2957795131)]
    return coord[0],coord[1]

df_his_location['LAT'],df_his_location['LONG'] = zip(*df_his_location.apply(change_to_loc, axis=1))

CPU times: user 5.25 s, sys: 0 ns, total: 5.25 s
Wall time: 42.4 s


In [ ]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(96772, 97524)

Multiple locations were present, so had to do treating

In [ ]:
%%time
def loc_treatment(group):
    group = group.reset_index(drop = True)
    if((group.LAT.nunique() > 1) | (group.LONG.nunique() > 1)):
        x = 0.0
        y = 0.0
        z = 0.0
        for i in range(len(group)):
            latitude = math.radians(float(group.LAT[i]))
            longitude = math.radians(float(group.LONG[i]))
            x += math.cos(latitude) * math.cos(longitude)
            y += math.cos(latitude) * math.sin(longitude)
            z += math.sin(latitude)
        total = len(group)
        x = x / total
        y = y / total
        z = z / total
        central_longitude = math.atan2(y, x)
        central_square_root = math.sqrt(x * x + y * y)
        central_latitude = math.atan2(z, central_square_root)
        group.LAT = math.degrees(central_latitude)
        group.LONG = math.degrees(central_longitude)
        return group
    else:
        return group

df_his_location = df_his_location.groupby(['INCIDENT_ID','STRCTUR_NO'], as_index = False).apply(loc_treatment)

CPU times: user 5min 6s, sys: 22.1 s, total: 5min 28s
Wall time: 37min 15s


In [ ]:
df_his_location = df_his_location[['INCIDENT_ID','STRCTUR_NO','LAT','LONG']].drop_duplicates()

In [ ]:
(df_his_location['INCIDENT_ID'].astype(str) + df_his_location['STRCTUR_NO'].astype(str)).nunique(), len(df_his_location)

(96772, 96772)

In [ ]:
df_his_location['LAT'] = df_his_location['LAT'].astype(float)
df_his_location['LONG'] = df_his_location['LONG'].astype(float)
df_his_location['LAT'].min(), df_his_location['LAT'].max()

(35.24263651397552, 39.98297582730674)

In [ ]:
df_his_location['LONG'].min(), df_his_location['LONG'].max()

(-86.7646286032229, -85.9158985730893)

In [ ]:
# function to add zone feature to the ads according to geo coordinates
def add_zone_feature(df):
    center_lat = 39.7684
    center_long = -86.1581
    zone = ''
    
    if(float(df['LAT']) < center_lat):
        if(float(df['LONG']) < center_long):
            zone = 'ZONE1'
        else:
            zone = 'ZONE2'
    else:
        if(float(df['LONG']) < center_long):
            zone = 'ZONE4'
        else:
            zone = 'ZONE3'
    
    return zone

df_his_location['ZONE'] = df_his_location.apply(add_zone_feature, axis=1)
print(df_his_location['ZONE'].unique())

['ZONE4' 'ZONE2' 'ZONE3' 'ZONE1']


In [ ]:
print(list(df_fac_cat.columns))

['INCIDENT_ID', 'STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG', 'CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 'FOUND_OPEN_OCCURN_FLG', 'PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG', 'REGULATOR_OCCURN

In [ ]:
df_fac_cat = pd.merge(df_fac_cat, df_his_location, on = ['INCIDENT_ID','STRCTUR_NO'], how = "left")

In [ ]:
print(df_fac_cat.shape)
df_fac_cat.head()

(94461, 67)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CAUSE_DESC,CLUE_DESC,OCCURN_DESC,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,LAT,LONG,ZONE
0,2000383864.00,494-B/403,1252.00,FUSE,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,O.H. EQUIPMENT\BAD ARRESTOR,PART LIGHTS - DIM/BRIGHT\PART OUT (RESET BREAK...,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD ARRESTOR,39.78,-86.27,ZONE4
1,2000383866.00,404-A/85,1958.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,39.83,-86.17,ZONE4
2,2000383867.00,675--/125,1701.00,3TBOH,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,PUBLIC\VANDALISM,POWER OUT - CUSTOMER\OUTAGE\BUSINESS,NONE\NONE,INDIANAPOLIS,NONE,NONE,None,PUBLIC,VANDALISM,39.71,-86.07,ZONE2
3,2000383871.00,465-A/25,1960.00,1TBOH,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,MISCELLANEOUS\CUSTOMER EQUIPMENT,DO NOT USE - IVR\POWER OUT,CUSTOMER EQUIP.\CUST. OWNED WIRE,INDIANAPOLIS,CUSTOMER EQUIP.,CUST. OWNED WIRE,None,MISCELLANEOUS,CUSTOMER EQUIPMENT,39.80,-86.17,ZONE4
4,2000383873.00,491XA/60,2702.00,FUSE,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,39.78,-86.32,ZONE4


In [ ]:
df_ads = pd.merge(df_numerical, df_fac_cat, on = ['INCIDENT_ID','STRCTUR_NO', 'CIRCT_ID', 'DNI_EQUIP_TYPE'], how = "inner")

In [ ]:
print(df_fac_cat.shape)
print(df_numerical.shape)
print(df_ads.shape)
df_ads.head()

(94461, 67)
(94523, 21)
(94523, 84)


,INCIDENT_ID,STRCTUR_NO,CIRCT_ID,DNI_EQUIP_TYPE,STORM_EVENT_FLG,EVENT,CUST_QTY,CALL_QTY,KEY_CUST_QTY,DOWNSTREAM_CUST_QTY,KVA_VAL,DOWNSTREAM_KVA_VAL,FAC_JOB_ID,ETR_DATETIME,CREATION_DATETIME,MAJ_OTG_ID,ENERGIZED_DATETIME,SUBST_ID,COMMENT_TEXT,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CAUSE_DESC,CLUE_DESC,OCCURN_DESC,CITY_NAM,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,LAT,LONG,ZONE
0,2000383864.00,494-B/403,1252.00,FUSE,0,BLUE SKY,5.00,1,0.00,5.00,433.30,433.30,2001309379,2007-01-01 11:46:14,2007-01-01 08:16:14,0.00,2007-01-01 10:04:24,125.00,contact # 484-1600 ext 804. gs,1,108.17,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,O.H. EQUIPMENT\BAD ARRESTOR,PART LIGHTS - DIM/BRIGHT\PART OUT (RESET BREAK...,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD ARRESTOR,39.78,-86.27,ZONE4
1,2000383866.00,404-A/85,1958.00,1TBOH,0,BLUE SKY,6.00,2,0.00,6.00,25.00,25.00,2001309376,2007-01-01 13:06:18,2007-01-01 09:36:18,0.00,2007-01-01 10:15:54,195.00,NaN,1,39.60,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,39.83,-86.17,ZONE4
2,2000383867.00,675--/125,1701.00,3TBOH,0,BLUE SKY,1.00,1,0.00,1.00,37.00,37.00,2001309377,2007-01-01 13:08:21,2007-01-01 09:38:00,0.00,2007-01-01 10:15:00,170.00,steve with a g a gas says cutouts open on bank...,1,37.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,PUBLIC\VANDALISM,POWER OUT - CUSTOMER\OUTAGE\BUSINESS,NONE\NONE,INDIANAPOLIS,NONE,NONE,None,PUBLIC,VANDALISM,39.71,-86.07,ZONE2
3,2000383871.00,465-A/25,1960.00,1TBOH,0,BLUE SKY,8.00,1,0.00,8.00,25.00,25.00,2001309382,2007-01-01 14:08:10,2007-01-01 10:38:10,0.00,2007-01-01 11:11:04,195.00,cusytomer says he beleives dead leg at top of ...,1,32.90,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,MISCELLANEOUS\CUSTOMER EQUIPMENT,DO NOT USE - IVR\POWER OUT,CUSTOMER EQUIP.\CUST. OWNED WIRE,INDIANAPOLIS,CUSTOMER EQUIP.,CUST. OWNED WIRE,None,MISCELLANEOUS,CUSTOMER EQUIPMENT,39.80,-86.17,ZONE4
4,2000383873.00,491XA/60,2702.00,FUSE,0,BLUE SKY,12.00,6,0.00,12.00,200.00,200.00,2001309386,2007-01-01 13:37:37,2007-01-01 11:07:37,0.00,2007-01-01 12:12:17,270.00,NaN,1,64.67,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ANIMAL\SQUIRREL,DO NOT USE - IVR\POWER OUT,FUSE\FUSE BLOWN,INDIANAPOLIS,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,39.78,-86.32,ZONE4


## **ADD NO OF OUTAGES FOR CLUE, CAUSE, OCCURN**

In [ ]:
df_ads['CREATION_DATETIME'] = pd.to_datetime(df_ads['CREATION_DATETIME'])
df_ads['Date'] = df_ads['CREATION_DATETIME'].dt.date

df_no_of_outages = df_ads.groupby(['Date'],as_index=False).agg({'POWER_OUT_CLUE_FLG' : 'sum', 'OPEN_DEVICE_CLUE_FLG' : 'sum', 'IVR_CLUE_FLG' : 'sum', 'ANIMAL_CAUSE_FLG' : 'sum',
                                                                'WIRE_OCCURN_FLG' : 'sum'})
df_no_of_outages.rename(columns = {'POWER_OUT_CLUE_FLG' : 'NO_OF_POWER_OUT_CLUE_PER_DAY', 'OPEN_DEVICE_CLUE_FLG' : 'NO_OF_OPEN_DEVICE_CLUE_PER_DAY',
                                   'IVR_CLUE_FLG' : 'NO_OF_IVR_CLUE_PER_DAY', 'ANIMAL_CAUSE_FLG' : 'NO_OF_ANIMAL_CAUSE_PER_DAY',
                                   'WIRE_OCCURN_FLG' : 'NO_OF_WIRE_OCCURN_PER_DAY'}, inplace=True)

df_no_of_outages.head()

,Date,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY
0,2007-01-01,7,0,5,2,1
1,2007-01-02,13,0,3,2,1
2,2007-01-03,18,0,7,4,2
3,2007-01-04,20,0,5,0,3
4,2007-01-05,18,0,5,0,1


In [ ]:
df_ads = df_ads[['FAC_JOB_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 
                 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM', 'LAT', 'LONG', 'ZONE', 'Date',
             'PRIM_OCCURN_DESC', 'SECO_OCCURN_DESC', 'TERT_OCCURN_DESC', 'PRIM_CAUSE_DESC', 'SECO_CAUSE_DESC', 'COMMENT_TEXT']]

df_ads.columns = ['OUTAGE_ID','INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 
         'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 
         'KVA_VAL', 'DAY_FLAG', 'TTR',
                  'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG','POWER_OUT_CLUE_FLG','OPEN_DEVICE_CLUE_FLG',
                'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG','IVR_CLUE_FLG','EQUIPMENT_CLUE_FLG','TRANSFORMER_CLUE_FLG',
             'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG','WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
             'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG','WEATHER__WIND_CAUSE_FLG',
             'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG','WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG',
             'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG','MISCELLANEOUS_CAUSE_FLG','NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 
              'NO_OUTAGE_CAUSE_FLG',
             'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 
             'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG','CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 
             'FOUND_OPEN_OCCURN_FLG','PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG',
             'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG','CAP_BANK_OCCURN_FLG','RECLOSER_OCCURN_FLG','OH_OCCURN_FLG',
             'PRIORITY_VAL_1.0','PRIORITY_VAL_2.0','PRIORITY_VAL_3.0','PRIORITY_VAL_5.0', 'CITY_NAM', 'LAT', 'LONG', 'ZONE', 'Date',
             'PRIM_OCCURN_DESC', 'SECO_OCCURN_DESC', 'TERT_OCCURN_DESC', 'PRIM_CAUSE_DESC', 'SECO_CAUSE_DESC', 'COMMENT_TEXT']

In [ ]:
df_ads = pd.merge(df_ads, df_no_of_outages, on=['Date'], how='left')

In [ ]:
df_ads.drop(['Date'],axis=1,inplace=True)

In [ ]:
print(df_ads.shape)
df_ads.head()

(94523, 84)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY
0,2001309379,2000383864.00,494-B/403,BLUE SKY,2007-01-01 08:16:14,2007-01-01 10:04:24,1252.00,FUSE,125.00,1,5.00,0.00,2007-01-01 11:46:14,5.00,433.30,433.30,1,108.17,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.78,-86.27,ZONE4,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD ARRESTOR,contact # 484-1600 ext 804. gs,7,0,5,2,1
1,2001309376,2000383866.00,404-A/85,BLUE SKY,2007-01-01 09:36:18,2007-01-01 10:15:54,1958.00,1TBOH,195.00,2,6.00,0.00,2007-01-01 13:06:18,6.00,25.00,25.00,1,39.60,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.83,-86.17,ZONE4,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,NaN,7,0,5,2,1
2,2001309377,2000383867.00,675--/125,BLUE SKY,2007-01-01 09:38:00,2007-01-01 10:15:00,1701.00,3TBOH,170.00,1,1.00,0.00,2007-01-01 13:08:21,1.00,37.00,37.00,1,37.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.71,-86.07,ZONE2,NONE,NONE,None,PUBLIC,VANDALISM,steve with a g a gas says cutouts open on bank...,7,0,5,2,1
3,2001309382,2000383871.00,465-A/25,BLUE SKY,2007-01-01 10:38:10,2007-01-01 11:11:04,1960.00,1TBOH,195.00,1,8.00,0.00,2007-01-01 14:08:10,8.00,25.00,25.00,1,32.90,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.80,-86.17,ZONE4,CUSTOMER EQUIP.,CUST. OWNED WIRE,None,MISCELLANEOUS,CUSTOMER EQUIPMENT,cusytomer says he beleives dead leg at top of ...,7,0,5,2,1
4,2001309386,2000383873.00,491XA/60,BLUE SKY,2007-01-01 11:07:37,2007-01-01 12:12:17,2702.00,FUSE,270.00,6,12.00,0.00,2007-01-01 13:37:37,12.00,200.00,200.00,1,64.67,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.78,-86.32,ZONE4,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,NaN,7,0,5,2,1


In [ ]:
ads = df_ads.copy(deep=True)
ads=ads[['OUTAGE_ID', 'LAT', 'LONG']]
ads.reset_index(drop=True,inplace=True)
ads.head()

,OUTAGE_ID,LAT,LONG
0,2001309379,39.78,-86.27
1,2001309376,39.83,-86.17
2,2001309377,39.71,-86.07
3,2001309382,39.80,-86.17
4,2001309386,39.78,-86.32


In [ ]:
ads['LAT'] = pd.to_numeric(ads['LAT'])
ads['LONG'] = pd.to_numeric(ads['LONG'])
print(ads.dtypes)
ads.head()

In [ ]:
ads['Marker1_LAT'] =  39.9613 
ads['Marker2_LAT'] = 39.8971
ads['Marker3_LAT'] = 39.9060
ads['Marker4_LAT'] = 39.9024
ads['Marker5_LAT'] = 39.8960
ads['Marker6_LAT'] = 39.8339
ads['Marker7_LAT'] = 39.8412
ads['Marker8_LAT'] = 39.8381
ads['Marker9_LAT'] = 39.8386
ads['Marker10_LAT'] = 39.7579
ads['Marker11_LAT'] = 39.7621
ads['Marker12_LAT'] = 39.7621
ads['Marker13_LAT'] = 39.7695
ads['Marker14_LAT'] = 39.6617
ads['Marker15_LAT'] = 39.6639
ads['Marker16_LAT'] = 39.6702
ads['Marker17_LAT'] = 39.6744
ads['Marker18_LAT'] = 39.5909
ads['Marker19_LAT'] = 39.5295
ads['Marker20_LAT'] = 39.5475

ads['Marker1_LONG'] = -86.4034 
ads['Marker2_LONG'] = -86.3045
ads['Marker3_LONG'] = -86.2001
ads['Marker4_LONG'] = -86.0738
ads['Marker5_LONG'] = -85.9783
ads['Marker6_LONG'] = -86.3155
ads['Marker7_LONG'] = -86.2056
ads['Marker8_LONG'] = -86.0985
ads['Marker9_LONG'] = -85.9811
ads['Marker10_LONG'] = -86.3155
ads['Marker11_LONG'] = -86.2042
ads['Marker12_LONG'] = -86.0923
ads['Marker13_LONG'] = -85.9708
ads['Marker14_LONG'] = -86.2935
ads['Marker15_LONG'] = -86.1823
ads['Marker16_LONG'] = -86.0669
ads['Marker17_LONG'] = -85.9557
ads['Marker18_LONG'] = -86.4212
ads['Marker19_LONG'] = -86.5874
ads['Marker20_LONG'] = -86.2743

In [ ]:
# calculate distance from 2 lat long 

def haversine(p1, p2):
    R = 6371     # earth radius in km
    p1 = [math.radians(v) for v in p1]
    p2 = [math.radians(v) for v in p2]

    d_lat = p2[0] - p1[0]
    d_lng = p2[1] - p1[1]
    a = math.pow(math.sin(d_lat / 2), 2) + math.cos(p1[0]) * math.cos(p2[0]) * math.pow(math.sin(d_lng / 2), 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c   # returns distance between p1 and p2 in km


In [ ]:
# calculate minimum distance

def minimum_distance(lat, long, marker1_lat, marker2_lat, marker3_lat, marker4_lat, marker5_lat, marker6_lat, marker7_lat, marker8_lat, marker9_lat, marker10_lat, marker11_lat,
                     marker12_lat, marker13_lat, marker14_lat, marker15_lat, marker16_lat, marker17_lat, marker18_lat, marker19_lat, marker20_lat, marker1_long, marker2_long,
                     marker3_long, marker4_long, marker5_long, marker6_long, marker7_long, marker8_long, marker9_long, marker10_long, marker11_long, marker12_long, marker13_long,
                     marker14_long, marker15_long, marker16_long, marker17_long, marker18_long, marker19_long, marker20_long):
    
    dist1 = haversine((lat,long), (marker1_lat, marker1_long))
    dist2 = haversine((lat,long), (marker2_lat, marker2_long))
    dist3 = haversine((lat,long), (marker3_lat, marker3_long))
    dist4 = haversine((lat,long), (marker4_lat, marker4_long))
    dist5 = haversine((lat,long), (marker5_lat, marker5_long))
    dist6 = haversine((lat,long), (marker6_lat, marker6_long))
    dist7 = haversine((lat,long), (marker7_lat, marker7_long))
    dist8 = haversine((lat,long), (marker8_lat, marker8_long))
    dist9 = haversine((lat,long), (marker9_lat, marker9_long))
    dist10 = haversine((lat,long), (marker10_lat, marker10_long))
    dist11 = haversine((lat,long), (marker11_lat, marker11_long))
    dist12 = haversine((lat,long), (marker12_lat, marker12_long))
    dist13 = haversine((lat,long), (marker13_lat, marker13_long))
    dist14 = haversine((lat,long), (marker14_lat, marker14_long))
    dist15 = haversine((lat,long), (marker15_lat, marker15_long))
    dist16 = haversine((lat,long), (marker16_lat, marker16_long))
    dist17 = haversine((lat,long), (marker17_lat, marker17_long))
    dist18 = haversine((lat,long), (marker18_lat, marker18_long))
    dist19 = haversine((lat,long), (marker19_lat, marker19_long))
    dist20 = haversine((lat,long), (marker20_lat, marker20_long))
    
    dist_list = [dist1, dist2, dist3, dist4, dist5, dist6, dist7, dist8, dist9, dist10, dist11, dist12, dist13, dist14, dist15, dist16, dist17, dist18, dist19, dist20]

    min_index, min_value = min(enumerate(dist_list), key=operator.itemgetter(1))
    
    if ( (math.isnan(lat)) | (math.isnan(long)) ):
        return None, None
    else :
        return min_value, min_index+1

In [ ]:
%%time
ads['Min_Distance'], ads['Marker_Location'] = zip(*ads.apply(lambda row: minimum_distance(row['LAT'], row['LONG'], row['Marker1_LAT'], row['Marker2_LAT'],
                                                            row['Marker3_LAT'], row['Marker4_LAT'], row['Marker5_LAT'], row['Marker6_LAT'],
                                                            row['Marker7_LAT'], row['Marker8_LAT'], row['Marker9_LAT'], row['Marker10_LAT'], 
                                                            row['Marker11_LAT'], row['Marker12_LAT'], row['Marker13_LAT'], row['Marker14_LAT'],
                                                            row['Marker15_LAT'], row['Marker16_LAT'], row['Marker17_LAT'], row['Marker18_LAT'],
                                                            row['Marker19_LAT'], row['Marker20_LAT'], row['Marker1_LONG'], row['Marker2_LONG'],
                                                            row['Marker3_LONG'], row['Marker4_LONG'], row['Marker5_LONG'], row['Marker6_LONG'], 
                                                            row['Marker7_LONG'], row['Marker8_LONG'], row['Marker9_LONG'], row['Marker10_LONG'],
                                                            row['Marker11_LONG'], row['Marker12_LONG'], row['Marker13_LONG'], row['Marker14_LONG'],
                                                            row['Marker15_LONG'], row['Marker16_LONG'], row['Marker17_LONG'], row['Marker18_LONG'], 
                                                            row['Marker19_LONG'], row['Marker20_LONG']),axis=1))

CPU times: user 37.8 s, sys: 0 ns, total: 37.8 s
Wall time: 5min 13s


In [ ]:
ads = ads[['OUTAGE_ID', 'LAT', 'LONG', 'Min_Distance', 'Marker_Location']]
ads.head()

,OUTAGE_ID,LAT,LONG,Min_Distance,Marker_Location
0,2001309379,39.78,-86.27,4.81,10
1,2001309376,39.83,-86.17,3.42,7
2,2001309377,39.71,-86.07,4.17,16
3,2001309382,39.80,-86.17,5.44,7
4,2001309386,39.78,-86.32,2.41,10


In [ ]:
ads['Marker_Location'] = 'Marker' + ads['Marker_Location'].astype(str)
print(ads.Marker_Location.unique())

['Marker10' 'Marker7' 'Marker16' 'Marker12' 'Marker3' 'Marker4' 'Marker2'
 'Marker15' 'Marker13' 'Marker9' 'Marker6' 'Marker11' 'Marker8' 'Marker5'
 'Marker19' 'Marker17' 'Marker14' 'Marker20' 'Marker18' 'Marker1'
 'MarkerNone']


In [ ]:
ads.drop(['LAT','LONG'],axis=1,inplace=True)
ads.head()
print(df_ads.shape)
df_ads = pd.merge(df_ads, ads, how='left', on=['OUTAGE_ID'])
print(df_ads.shape)

(94523, 84)
(94647, 86)


In [ ]:
print(list(df_ads.columns))

['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER_OCCURN_FLG', 

## **ADD CYCLICITY ACCORDING TO HOUR**

In [ ]:
df_ads['Hour'] = df_ads['CREATION_DATETIME'].dt.hour
print(df_ads['Hour'].unique())

[ 8  9 10 11 15 20  0  6  7 13 14 19  3 12 17 22 16 21  2 18 23  4  1  5]


In [ ]:
df_ads['Hour_Sin'] = np.sin(df_ads.Hour*(2.0*np.pi/24))
df_ads['Hour_Cos'] = np.cos(df_ads.Hour*(2.0*np.pi/24))

In [ ]:
df_ads.drop(['Hour'],axis=1,inplace=True)

## **MAJ_OTG_ID ADDTION**
- Not valid for blue-sky + storm analytical dataset

In [ ]:
# ads_final = pd.merge(df_ads, maj_otg_df, on=['INCIDENT_ID', 'STRCTUR_NO'], how='left')
# print(ads_final.shape)
# ads_final.head()

In [ ]:
# create a copy of the previous dataframe so that rerunning of code can be avaoided
ads_final = df_ads.copy(deep=True)

## **ADD SUBSEQUENT OUTAGES**

In [ ]:
ads_final['Date'] = ads_final.CREATION_DATETIME.dt.date
ads_final['RANK_SUBSEQUENT_OUTAGES'] = ads_final.groupby('Date')['CREATION_DATETIME'].rank(method='dense', ascending=True)
ads_final.drop(['Date','Min_Distance'],axis=1,inplace=True)

In [ ]:
print(ads_final.shape)
ads_final.head()

(94647, 88)


,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,PRIM_OCCURN_DESC,SECO_OCCURN_DESC,TERT_OCCURN_DESC,PRIM_CAUSE_DESC,SECO_CAUSE_DESC,COMMENT_TEXT,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES
0,2001309379,2000383864.00,494-B/403,BLUE SKY,2007-01-01 08:16:14,2007-01-01 10:04:24,1252.00,FUSE,125.00,1,5.00,0.00,2007-01-01 11:46:14,5.00,433.30,433.30,1,108.17,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.78,-86.27,ZONE4,FUSE,FUSE BLOWN,None,O.H. EQUIPMENT,BAD ARRESTOR,contact # 484-1600 ext 804. gs,7,0,5,2,1,Marker10,0.87,-0.50,1.00
1,2001309376,2000383866.00,404-A/85,BLUE SKY,2007-01-01 09:36:18,2007-01-01 10:15:54,1958.00,1TBOH,195.00,2,6.00,0.00,2007-01-01 13:06:18,6.00,25.00,25.00,1,39.60,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.83,-86.17,ZONE4,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,NaN,7,0,5,2,1,Marker7,0.71,-0.71,2.00
2,2001309377,2000383867.00,675--/125,BLUE SKY,2007-01-01 09:38:00,2007-01-01 10:15:00,1701.00,3TBOH,170.00,1,1.00,0.00,2007-01-01 13:08:21,1.00,37.00,37.00,1,37.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.71,-86.07,ZONE2,NONE,NONE,None,PUBLIC,VANDALISM,steve with a g a gas says cutouts open on bank...,7,0,5,2,1,Marker16,0.71,-0.71,3.00
3,2001309382,2000383871.00,465-A/25,BLUE SKY,2007-01-01 10:38:10,2007-01-01 11:11:04,1960.00,1TBOH,195.00,1,8.00,0.00,2007-01-01 14:08:10,8.00,25.00,25.00,1,32.90,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.80,-86.17,ZONE4,CUSTOMER EQUIP.,CUST. OWNED WIRE,None,MISCELLANEOUS,CUSTOMER EQUIPMENT,cusytomer says he beleives dead leg at top of ...,7,0,5,2,1,Marker7,0.50,-0.87,4.00
4,2001309386,2000383873.00,491XA/60,BLUE SKY,2007-01-01 11:07:37,2007-01-01 12:12:17,2702.00,FUSE,270.00,6,12.00,0.00,2007-01-01 13:37:37,12.00,200.00,200.00,1,64.67,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,INDIANAPOLIS,39.78,-86.32,ZONE4,FUSE,FUSE BLOWN,None,ANIMAL,SQUIRREL,NaN,7,0,5,2,1,Marker10,0.26,-0.97,5.00


## **ADDING LIVE OUTAGE**

In [ ]:
def count_outage(group):
    group = group.reset_index(drop = True)
    group['LIVE_OUTAGE'] = len(ads_final[(ads_final.CREATION_DATETIME < group.CREATION_DATETIME[0]) & (ads_final.ENERGIZED_DATETIME > group.CREATION_DATETIME[0])])
    return group

def grouping_fn(df):
    liveoutage = df.groupby(['OUTAGE_ID'], as_index=False).apply(count_outage)
    return liveoutage

if __name__ == '__main__':
    starttime = time.time()
    with Pool(30) as p:
            live_outage = p.map(grouping_fn, [ads_final[:5000], ads_final[5000:10000], ads_final[10000:15000],
                                  ads_final[15000:20000], ads_final[20000:25000], ads_final[25000:30000],
                                  ads_final[30000:35000], ads_final[35000:40000], ads_final[40000:50000],
                                  ads_final[50000:55000], ads_final[55000:60000], ads_final[60000:65000],
                                   ads_final[65000:70000], ads_final[70000:75000], ads_final[75000:80000],
                                  ads_final[80000:90000], ads_final[90000:100000], ads_final[100000:105000],
                                  ads_final[105000:]])
    print('That took {} seconds'.format(time.time() - starttime))

That took 818.175573348999 seconds


In [ ]:
ads_final=pd.concat(live_outage)

In [ ]:
print(ads_final.shape)
ads_final.reset_index(drop=True,inplace=True)
ads_final.head()

In [ ]:
ads_final.head()

## **OUTAGE FEATURES**

In [ ]:
def count_outage_minutes(group):
    group = group.reset_index(drop = True)
    df_temp = ads_final[['OUTAGE_ID','CREATION_DATETIME']]
    df_temp['minutes'] = (group['CREATION_DATETIME'][0] - ads_final['CREATION_DATETIME']).dt.total_seconds().div(60)
    df_temp = df_temp[df_temp.minutes > 0]
    group['Outages_in_last_1hr'] = len(df_temp[df_temp.minutes <= 60])
    group['Outages_in_last_2hr'] = len(df_temp[df_temp.minutes <= 120])
    group['Outages_in_last_3hr'] = len(df_temp[df_temp.minutes <= 180])
    group['Outages_in_last_4hr'] = len(df_temp[df_temp.minutes <= 240])
    group['Outages_in_last_5hr'] = len(df_temp[df_temp.minutes <= 300])
    group['Outages_in_last_6hr'] = len(df_temp[df_temp.minutes <= 360])
    group['Outages_in_last_7hr'] = len(df_temp[df_temp.minutes <= 420])
    group['Outages_in_last_8hr'] = len(df_temp[df_temp.minutes <= 480])
    group['Outages_in_last_9hr'] = len(df_temp[df_temp.minutes <= 540])
    group['Outages_in_last_10hr'] = len(df_temp[df_temp.minutes <= 600])
    return group

def grouping_fn_minutes(df):
    liveoutage = df.groupby(['OUTAGE_ID'], as_index=False).apply(count_outage_minutes)
    return liveoutage

if __name__ == '__main__':
    starttime = time.time()
    with Pool(30) as p:
            live_outage_minutes = p.map(grouping_fn_minutes, [ads_final[:5000], ads_final[5000:10000], ads_final[10000:15000],
                                  ads_final[15000:20000], ads_final[20000:25000], ads_final[25000:30000],
                                  ads_final[30000:35000], ads_final[35000:40000], ads_final[40000:50000],
                                  ads_final[50000:55000], ads_final[55000:60000], ads_final[60000:65000],
                                  ads_final[65000:70000], ads_final[70000:75000], ads_final[75000:80000],
                                  ads_final[80000:90000], ads_final[90000:100000], ads_final[100000:105000],
                                  ads_final[105000:]])
    print('That took {} seconds'.format(time.time() - starttime))

In [ ]:
ads_final=pd.concat(live_outage_minutes)

In [ ]:
print(ads_final.shape)
ads_final.reset_index(drop=True,inplace=True)
ads_final.head()

In [ ]:
df_copy = ads_final.copy(deep=True)
# ads_final = df_copy.copy(deep=True)

In [ ]:
print(df_copy.shape)
print(ads_final.shape)

## **Day of the week features**

In [ ]:
ads_final['Date'] = ads_final.CREATION_DATETIME.dt.date
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
ads_final['Weekday'] = ads_final['Date'].apply(lambda x: x.weekday()).apply(lambda x: days[x])

ads_final['Weekend_flag'] = ads_final['Weekday'].apply(lambda x: True if (x == 'Saturday') | (x == 'Sunday') else False)
ads_final.drop(['Date'],axis=1,inplace=True)

In [ ]:
print(list(ads_final.columns))
print(ads_final.shape)

## **Priority Queuing Feature**
1. Rank based on simple customer quantity as mentioned by Eric (live rankings to be followed, numerical feature) <br>

In [ ]:
ads_final.sort_values(by = ['CREATION_DATETIME'], inplace=True)
ads_final.reset_index(drop=True, inplace=True)

In [ ]:
def create_groups_based_on_live_outages(live):
    list_group_no = []
    group = 0
    
    for i in range(len(live)):
        if live.LIVE_OUTAGE[i] == 0  :
            group = group + 1
            list_group_no.append(group)
        else :
            list_group_no.append(group)
    
    return list_group_no

ads_final['Live_outage_group'] = create_groups_based_on_live_outages(ads_final)

In [ ]:
ads_final['Priority_Customer_Qty'] = ads_final.groupby(['Live_outage_group'])['DOWNSTREAM_CUST_QTY'].rank(method='dense', ascending=False)

In [ ]:
print(ads_final.shape)

2. Rank based on the factor of distance from centroid and customer quantity (live rankings to be followed, approach #2, numerical feature) <br>

In [ ]:
df_v1 = ads_final.groupby(['Live_outage_group'],as_index=False).agg({'LAT' : 'sum', 'LONG' : 'sum', 'LIVE_OUTAGE' : 'count'})
df_v1['Center_LAT'] = (df_v1.LAT)/(df_v1.LIVE_OUTAGE)
df_v1['Center_LONG'] = (df_v1.LONG)/(df_v1.LIVE_OUTAGE)
df_v1.head()

In [ ]:
df_v1.drop(['LAT', 'LONG', 'LIVE_OUTAGE'], axis=1, inplace=True)
ads_final_v1 = pd.merge(ads_final, df_v1, how='left', on='Live_outage_group')

In [ ]:
print(ads_final_v1.shape)
# ads_final_v1.head(5)

In [ ]:
def cal_distance_from_center_lat_long(lat, long, center_lat, center_long):
    if ((math.isnan(lat)) | (math.isnan(long)) | (math.isnan(center_lat)) | (math.isnan(center_long))):
        return None
    else :
        coords1 = [lat,long]
        coords2 = [center_lat, center_long]
        return (geopy.distance.distance(coords1, coords2).miles)

In [ ]:
ads_final_v1['Dis_From_Live_Centriod'] = ads_final_v1.apply(lambda x: cal_distance_from_center_lat_long(x['LAT'], x['LONG'], x['Center_LAT'], x['Center_LONG']),axis=1)
ads_final_v1['Dis_From_Live_Centriod'] = ads_final_v1['Dis_From_Live_Centriod'].apply(pd.to_numeric, errors='coerce')

In [ ]:
ads_final_v1['Dis_From_Live_Centriod_div_Cust_qty'] = (ads_final_v1['Dis_From_Live_Centriod']) / (ads_final_v1['DOWNSTREAM_CUST_QTY'])
ads_final_v1['Priority_Dist_Customer_Qty'] = ads_final_v1.groupby(['Live_outage_group'])['Dis_From_Live_Centriod_div_Cust_qty'].rank(method='max', ascending=True)

In [ ]:
ads_final_v1.drop(['Center_LAT', 'Center_LONG', 'Dis_From_Live_Centriod'], axis=1, inplace=True)

In [ ]:
print(ads_final_v1.shape)

## **Add dispatch area location**

In [ ]:
def cal_distance_from_dipatch_area(lat, long):
    
    if ((math.isnan(lat)) | (math.isnan(long))):
        return None, None
    else :
        coords1 = [lat,long]
        dist_34 = geopy.distance.distance(coords1, [39.8802, -86.2324]).miles
        dist_arl = geopy.distance.distance(coords1, [39.8802, -86.0854]).miles
        dist_mill = geopy.distance.distance(coords1, [39.7880, -86.2296]).miles
        dist_english = geopy.distance.distance(coords1, [39.7880, -86.0868]).miles
        dist_wii = geopy.distance.distance(coords1, [39.7003, -86.2303]).miles
        dist_south = geopy.distance.distance(coords1, [39.7003, -86.0834]).miles
    
        dist_list = [dist_34, dist_arl, dist_mill, dist_english, dist_wii, dist_south]

        min_index, min_value = min(enumerate(dist_list), key=operator.itemgetter(1))
    
        return min_value, min_index+1

In [ ]:
ads_final_v1['Min_Distance'], ads_final_v1['Grid'] = zip(*ads_final_v1.apply(lambda row: cal_distance_from_dipatch_area(row['LAT'], row['LONG']),axis=1))

In [ ]:
def map_grid_to_location(row):
    
    if row==1:
        return '34th'
    elif row==2:
        return 'ARL.'
    elif row==3:
        return 'MILL'
    elif row==4:
        return 'ENGLISH'
    elif row==5:
        return 'W.I.'
    elif row==6:
        return 'SOUTH'
    else :
        return "NO_LOCATION"

In [ ]:
ads_final_v1['Dispatch_Location'] = ads_final_v1.apply(lambda row: map_grid_to_location(row['Grid']),axis=1)

In [ ]:
ads_final_v1.drop(['Min_Distance', 'Grid'], axis=1, inplace=True)

In [ ]:
ads_final_v1.head()

## **Weather data addition**

In [ ]:
newdf_ws = pd.read_csv("gs://aes-datahub-0001-raw/Weather/weather_source/USA/Indianapolis/Historical_Data/weather_dump_clusters.csv")

In [ ]:
len(newdf_ws)

In [ ]:
columns = ['DAY_FLAG','POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG',
           'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG','OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG',
           'PUBLIC_CAUSE_FLG','WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG','CUST_REQUEST_CAUSE_FLG', 
           'WEATHER__FLOOD_CAUSE_FLG','STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG','TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG',
          'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG','TRANSFORMER_OCCURN_FLG', 'METER_OCCURN_FLG', 'SERVICE_OCCURN_FLG',
           'CABLE_OCCURN_FLG', 'ST_OCCURN_FLG', 'FIRE_OCCURN_FLG', 'FOUND_OPEN_OCCURN_FLG', 'PUBLIC_SAFETY_OCCURN_FLG', 'WIRE_OCCURN_FLG', 'SWITCH_OCCURN_FLG', 
           'REGULATOR_OCCURN_FLG', 'CUTOUT_OCCURN_FLG', 'CAP_BANK_OCCURN_FLG', 'RECLOSER_OCCURN_FLG', 'OH_OCCURN_FLG', 'PRIORITY_VAL_1.0', 'PRIORITY_VAL_2.0',
           'PRIORITY_VAL_3.0', 'PRIORITY_VAL_5.0']
for i in columns:
    ads_final_v1[i] = ads_final_v1[i].apply(lambda x: True if x==1 else False)

In [ ]:
newdf_ws['timestamp'] = pd.to_datetime(newdf_ws['timestamp'].str[:10]).dt.date.astype(str)
ads_final_v1['Date'] = pd.to_datetime(ads_final_v1['CREATION_DATETIME']).dt.date.astype(str)

In [ ]:
newdf_ws['Location'] = newdf_ws['Location'].str.replace(" ","")

In [ ]:
newdf_ws = newdf_ws.drop( columns = ['X'])

In [ ]:
len(newdf_ws), len(newdf_ws.drop_duplicates())

## **ADDING WEATHER FEATURES**

In [ ]:
# removing unwanted columns

newdf_ws = newdf_ws.drop(['snowDepth'], axis = 1)
unwanted = newdf_ws.columns[newdf_ws.columns.str.startswith('presTend')]
newdf_ws = newdf_ws.drop(unwanted, axis=1)

In [ ]:
print(list(newdf_ws.columns))

In [ ]:
## Add range for columns with negative values 

newdf_ws['tempRange'] = newdf_ws['tempMax'] - newdf_ws['tempMin']
newdf_ws['windSpdRange'] = newdf_ws['windSpdMax'] - newdf_ws['windSpdMin']
newdf_ws['sfcPresRange'] = newdf_ws['sfcPresMax'] - newdf_ws['sfcPresMin']
newdf_ws['cldCvrRange'] = newdf_ws['cldCvrMax'] - newdf_ws['cldCvrMin']
newdf_ws['relHumRange'] = newdf_ws['relHumMax'] - newdf_ws['relHumMin']

In [ ]:
## Add ratio for columns which dont have negative values 

newdf_ws['relHumRatio'] = newdf_ws['relHumMax'] / newdf_ws['relHumMin']
newdf_ws['sfcPresRatio'] = newdf_ws['sfcPresMax'] / newdf_ws['sfcPresMin']

In [ ]:
newdf_ws = newdf_ws.replace([np.inf, -np.inf], np.nan)
nulls = newdf_ws.isnull().sum()

df_nulls = pd.DataFrame({'Feature': nulls.index, 'VALUES': nulls.values})
df_nulls[df_nulls.VALUES>=1]

In [ ]:
ads_df = pd.merge(ads_final_v1, newdf_ws,left_on = ['Date','Marker_Location'],right_on = ['timestamp','Location'],how = "left")

In [ ]:
print(ads_df.shape[0])
print(len(ads_df.drop_duplicates()))

In [ ]:
ads_df.drop_duplicates(keep='first', inplace=True)
ads_df.reset_index(drop=True,inplace=True)
print(ads_df.shape)

In [ ]:
# removing unwanted columns for the final data frame

ads_df = ads_df.drop(['DOWNSTREAM_KVA_VAL','KVA_VAL','Date','latitude','longitude','timestamp', 'radSolarMin','Location'], axis = 1)

## **ADD ACTIVE NO OF TRUCKS PER DAY**

In [ ]:
# read d16_memb_activity table from GCS landing bucket
d16_memb_activity = pd.read_csv(r'gs://aes-datahub-0001-landing/CAD/IPL/CAD.D16_MEMB_ACTIVITY.csv', sep = ";", encoding = "ISO-8859-1")
d16_memb_activity['Date'] = pd.to_datetime(d16_memb_activity['D16_DATETIME_KEY'].astype(str).str[:8])

# filter for DIST.OPS and DUTY.IN
# d16_memb_activity = d16_memb_activity[(d16_memb_activity.AGENCY_CODE == 'DIST.OPS') & (d16_memb_activity.D16S_CAD_ACTION == 'DUTY-IN')]
d16_memb_activity = d16_memb_activity[(d16_memb_activity.D16S_CAD_ACTION == 'DUTY-IN')]

active_crews = d16_memb_activity.groupby(['Date'],as_index=False).agg({'UNIT' : pd.Series.nunique})
active_crews_unique = d16_memb_activity.groupby(['Date'],as_index=False).agg({'UNIT' : pd.Series.unique})


active_crews.rename(columns={"UNIT":"Active_no_of_trucks_per_day"},inplace=True)

joined = pd.merge(active_crews, active_crews_unique, on=['Date'], how='left')

In [ ]:
del d16_memb_activity

ads_df['Date'] = ads_df['CREATION_DATETIME'].dt.date
ads_df['Date'] = pd.to_datetime(ads_df['Date'])
joined['Date'] = pd.to_datetime(joined['Date'])

ads_df = pd.merge(ads_df, joined, how='left', on=['Date'])
ads_df.drop(['Date'], axis=1, inplace=True)
print(ads_df.shape)

In [ ]:
ads_df.reset_index(drop=True, inplace=True)
print(ads_df.shape)

In [ ]:
ads_df.dropna(axis=0,subset=['LAT','LONG'], inplace=True)
ads_df.drop(['clusters','profile'],axis=1,inplace=True)

In [ ]:
print(ads_df.shape)

## **Add Repair Remarks from HIS_CREW_ACTION**

In [ ]:
df_crew = pd.read_csv('gs://aes-datahub-0001-raw/OMS/2002-2020/IPL/HIS_CREW_ACTION_IPL.csv', sep='|', encoding = "ISO-8859-1")

In [ ]:
final_merged_ads = pd.merge(ads_df, df_crew[['INCIDENT_ID', 'REPAIR_REMARK']], on=['INCIDENT_ID'], how='left')
final_merged_ads.head()

In [ ]:
final_merged_ads.isnull().sum()

In [ ]:
final_merged_ads.dtypes

In [ ]:
final_merged_ads['EVENT'].value_counts()

### **Major Storms detection & removal**

In [ ]:
final_merged_ads.to_csv('gs://aes-analytics-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_BASECREW_11232020.csv')

In [ ]:
ads_df['Month'] = ads_df['CREATION_DATETIME'].dt.month
ads_df['Year'] = ads_df['CREATION_DATETIME'].dt.year

ads_df['Period'] = ads_df.Year.astype(str) + "-" + ads_df.Month.astype(str)

df_outages = ads_df.groupby(['Period'],as_index=False).agg({'OUTAGE_ID' : 'count'})
df_outages.rename({'OUTAGE_ID' : 'No_of_outages'}, axis=1, inplace=True)
df_outages_v1 = df_outages[df_outages.No_of_outages>1200]

In [ ]:
sns.set(style="white")
sns.set(rc={'figure.figsize':(26,26)})
ax = sns.barplot(x="No_of_outages", y="Period", data=df_outages)

In [ ]:
df_outages['Year'] = df_outages.Period.apply(lambda x: '2019' if (x.lower().find('2019') != -1) else 0)

In [ ]:
sns.set(style="white")
sns.set(rc={'figure.figsize':(18,10)})
ax = sns.barplot(x="No_of_outages", y="Period", data=df_outages[df_outages['Year'] == '2019'])

In [ ]:
sns.set(style="white")
sns.set(rc={'figure.figsize':(26,10)})
ax = sns.barplot(x="No_of_outages", y="Period", data=df_outages_v1)

**2008 - 06**
- The tornado outbreak sequence of May 22–31, 2008 was a series of tornado outbreaks affecting the central plains of the United States from May 22 to May 31 of 2008. <br>
- It was also one of the largest continuous tornado outbreaks on record. <br>
Link - https://www.weather.gov/ind/may302008_tor <br>

**2011 - 06**
-The tornado outbreak sequence of April 19–24, 2011 was an extended period of significant tornado activity that began on April 19th, 2011 and ended on April 24, 2011. 
Link - https://en.wikipedia.org/wiki/Tornado_outbreak_sequence_of_April_19%E2%80%9324,_2011

**2015 - 07**
- Waves of showers and thunderstorms moved across central Indiana on July 13 and into the early morning of July 14. <br>
- Upper level features interacted with a surface front and plenty of moisture and instability to generate severe weather and heavy rains/flooding. <br>
- Two tornadoes occurred during the evening of the 13th. One tornado was in Warren County, and the other was in Putnam County. <br>
- Widespread tree and power line damage occurred across central Indiana, and some locations received over 4 inches of rain. <br>
- A few locations had hail as well. <br>
- The heavy rain led to flooding in many areas. <br>
Link - https://www.weather.gov/ind/july13142015severe <br>

**2016 - 06**
- An upper level system worked with a hot and unstable atmosphere to produce strong to severe thunderstorms across central Indiana during the afternoon and early evening of June 15. <br>
- Damaging winds and heavy rain/flooding were the main impacts. <br>
Link - https://www.weather.gov/ind/june152016severe <br>

### **Outlier Detection**
In statistics, an outlier is an observation point that is distant from other observations.

### **Discovering outliers with visualization tools**
- In descriptive statistics, a box plot is a method for graphically depicting groups of numerical data through their quartiles. <br>
- Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles, hence the terms box-and-whisker plot and box-and-whisker diagram. <br>
- Outliers may be plotted as individual points. <br>

In [ ]:
plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.boxplot(ads_df.TTR, palette = ['#81cb29'])
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 7000))
plt.plot()
print("Mean TTR", round(ads_df.TTR.mean(),2))
print("Median TTR", round(ads_df.TTR.median(),2))

plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.distplot(ads_df.TTR)
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 7000))
plt.plot()

In [ ]:
ads_df['Date'] = ads_df['CREATION_DATETIME'].dt.date
ads_df['Date'] = pd.to_datetime(ads_df['Date'])

In [ ]:
# ads_df = ads_df[~((ads_df['Date'].dt.year == 2008) & (ads_df['Date'].dt.month == 6))]
# ads_df = ads_df[~((ads_df['Date'].dt.year == 2011) & (ads_df['Date'].dt.month == 6))]
# ads_df = ads_df[~((ads_df['Date'].dt.year == 2015) & (ads_df['Date'].dt.month == 7))]
# ads_df = ads_df[~((ads_df['Date'].dt.year == 2016) & (ads_df['Date'].dt.month == 6))]

In [ ]:
print(ads_df.shape)
ads_df.head()

In [ ]:
plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.boxplot(ads_df.TTR, palette = ['#81cb29'])
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 7000))
plt.plot()
print("Mean TTR", round(ads_df.TTR.mean(),2))
print("Median TTR", round(ads_df.TTR.median(),2))

plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.distplot(ads_df.TTR)
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 7000))
plt.plot()

### **IQR score**
- The interquartile range (IQR), also called the midspread or middle 50%, or technically H-spread, is a measure of statistical dispersion, being equal to the difference between 95th and 5th percentiles, or between upper and lower quartiles, IQR = Q3 − Q1. <br>
- In other words, the IQR is the first quartile subtracted from the third quartile; these quartiles can be clearly seen on a box plot on the data. <br>
- It is a measure of the dispersion similar to standard deviation or variance, but is much more robust against outliers. <br>

In [ ]:
q_low = ads_df["TTR"].quantile(0.01)
q_hi  = ads_df["TTR"].quantile(0.99)

df_ipl_ads_v1 = ads_df[(ads_df["TTR"] < q_hi) & (ads_df["TTR"] > q_low)]
print(ads_df.shape)
print(df_ipl_ads_v1.shape)

In [ ]:
plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.boxplot(df_ipl_ads_v1.TTR, palette = ['#81cb29'])
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 3000))
plt.plot()
print("Mean TTR", round(df_ipl_ads_v1.TTR.mean(),2))
print("Median TTR", round(df_ipl_ads_v1.TTR.median(),2))

plt.figure(num = None,figsize=(16, 5),dpi=80,facecolor='w',edgecolor='k')
ax = sns.distplot(df_ipl_ads_v1.TTR)
ax.set_title('TTR distribution for Blue-sky + Storm events')
ax.set(xlim=(0, 3000))
plt.plot()

In [ ]:
df_ipl_ads_v1.reset_index(drop=True, inplace=True)
df_ipl_ads_v1.head()

In [ ]:
df_ipl_ads_v1.to_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_V10.4_IQR_FULL.csv',index=False)

### **Drop columns which will not be used for modelling purpose**

In [ ]:
# df_ipl_ads_v1_storm = df_ipl_ads_v1.copy(deep=True)
# df_ipl_ads_v1_storm = df_ipl_ads_v1_storm[df_ipl_ads_v1_storm.EVENT=='STORM']

# ads_df_storm = ads_df.copy(deep=True)
# ads_df_storm = ads_df_storm[ads_df_storm.EVENT=='STORM']

In [ ]:
df_ipl_ads_v1.drop(['OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'SUBST_ID', 
                    'ETR_DATETIME', 'CUST_QTY', 'LAT', 'LONG', 'LIVE_OUTAGE', 'Live_outage_group', 'Marker_Location', 'Dis_From_Live_Centriod_div_Cust_qty',
                    'Active_no_of_trucks_per_day', 'Month', 'Year', 'Period'], axis=1, inplace=True)

In [ ]:
# split into train test sets
# OMS_IPL_OUTAGE_DATASET_V10_4_TRAIN_IQR, OMS_IPL_OUTAGE_DATASET_V10_4_VALIDATION_IQR = train_test_split(df_ipl_ads_v1, test_size=0.20, random_state=369)

In [ ]:
# split into train test sets
# OMS_IPL_OUTAGE_DATASET_V10_4_TRAIN, OMS_IPL_OUTAGE_DATASET_V10_4_VALIDATION = train_test_split(ads_df, test_size=0.20, random_state=369)

In [ ]:
print(df_ipl_ads_v1.shape)
# print(df_ipl_ads_v1_storm.shape)
# print('\n')
# print(ads_df.shape)
# print(ads_df_storm.shape)

In [ ]:
print(list(df_ipl_ads_v1.columns))

## **WRITE TO CSV**

In [ ]:
df_ipl_ads_v1.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_V10.4_IQR_09152020.csv",index=False)
# df_ipl_ads_v1_storm.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_V10.4_Storm_IQR_08092020.csv",index=False)

# ads_df.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_V10.4_MAJOR_STORMS_08092020.csv",index=False)
# ads_df_storm.to_csv("gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_V10.4_Storm_MAJOR_STORMS_08092020.csv",index=False)